# Create Slice for Fabric Rack
This is the slice creation for the ```Routing with IPv4``` tutorial. This is the first step of the assignment, to create the slice on fabric using fablib.
 


## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 3 nodes for the IPv4 Assignment.
### 1.1 Reserve Resources
In the 'EDUKY' site, we will reserve a set of 3 nodes arranged in a triangle. Each node will have the following specifications: 1 CPU core, 2GB of RAM, and 10GB of storage capacity. The nodes will each have 2 network cards (NICs) that allow connection to its immediate neighbors. All nodes will be preloaded with an 'Ubuntu' Linux OS. The networks allocated will be "LAN_AB","LAN_BC" &"LAN_CA" using the corresponding interface ports from each node. Upon submission, the slice will be named 'IPv4_Routing'.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "IPv4_Routing"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    #Nodes
    ND_A = slice.add_node(name="ND_A", site=site)
    ND_A.set_capacities(cores=cores, ram=ram, disk=disk)
    ND_A.set_image(image)
    Ap1 = ND_A.add_component(model=nicmodel, name="Cn1").get_interfaces()[0] 
    Ap2 = ND_A.add_component(model=nicmodel, name="Cn2").get_interfaces()[0] 
    
    ND_B = slice.add_node(name="ND_B", site=site)
    ND_B.set_capacities(cores=cores, ram=ram, disk=disk)
    ND_B.set_image(image)
    Bp1 = ND_B.add_component(model=nicmodel, name="Cn1").get_interfaces()[0] 
    Bp2 = ND_B.add_component(model=nicmodel, name="Cn2").get_interfaces()[0] 

    ND_C = slice.add_node(name="ND_C", site=site)
    ND_C.set_capacities(cores=cores, ram=ram, disk=disk)
    ND_C.set_image(image)
    Cp1 = ND_C.add_component(model=nicmodel, name="Cn1").get_interfaces()[0] 
    Cp2 = ND_C.add_component(model=nicmodel, name="Cn2").get_interfaces()[0] 
    

    #Network
    lan1 = slice.add_l2network(name="Lan_AB", interfaces=[Ap2,Bp1])
    lan2 = slice.add_l2network(name="Lan_BC", interfaces=[Bp2,Cp1])
    lan3 = slice.add_l2network(name="Lan_CA", interfaces=[Cp2,Ap1])
    
    #Submit Slice Request
    slice.submit()
    
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
The network configuration part of this slice will connect the nodes with specific IPs so that they are able to communicate with each other. The slice will use the network name to find the corresponding devices (NIC cards) used in the slice creation, such as "LAN_AB", "LAN_BC", and "LAN_CA". This slice contains three subnetworks in IPv4: 192.168.1.0/24 (between node A & B, "LAN_AB"), 192.168.2.0/24 (between node B & C, "LAN_BC"), and 192.168.3.0/24 (between node C & A, "LAN_CA").

Each node will be given an IP address matching their neighbor's subnet. For example, under node 'A', the matching subnet to 'C' under "LAN_CA" is 192.168.3.0/24. This means that an available IP would be '192.168.3.10', which implies that C's IP would have to be 192.168.3.12 inside this subnet.

After providing the IPs to the corresponding devices, the slice will verify that all interfaces are up and execute "ip a" to output the connections that are active. Then, we will advertise the additional routes so that the nodes are aware of the active connections using the 'route' command.

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    NDA = slice.get_node('ND_A')
    NDB = slice.get_node('ND_B')
    NDC = slice.get_node('ND_C')
    subnet1 = IPv4Network("192.168.1.0/24")
    subnet2 = IPv4Network("192.168.2.0/24")
    subnet3 = IPv4Network("192.168.3.0/24")

    Ctrl_Network = 'Controll_Conn'
    
    #NDA
    A_iface = NDA.get_interface(network_name='Lan_AB')  
    A_iface.ip_addr_add(addr='192.168.1.10', subnet=subnet1)
    A_iface = NDA.get_interface(network_name='Lan_CA')  
    A_iface.ip_addr_add(addr='192.168.3.10', subnet=subnet3)
    
    #NDB
    B_iface = NDB.get_interface(network_name='Lan_AB')  
    B_iface.ip_addr_add(addr='192.168.1.11', subnet=subnet1)
    B_iface = NDB.get_interface(network_name='Lan_BC')  
    B_iface.ip_addr_add(addr='192.168.2.11', subnet=subnet2)
    
    #NDC
    C_iface = NDC.get_interface(network_name='Lan_BC')  
    C_iface.ip_addr_add(addr='192.168.2.12', subnet=subnet2) 
    C_iface = NDC.get_interface(network_name='Lan_CA')  
    C_iface.ip_addr_add(addr='192.168.3.12', subnet=subnet3)       

    for node in slice.get_nodes():  
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")      
        node.execute("sudo sysctl -w net.ipv4.ip_forward=1")
    
except Exception as e:
    print(f"Fail: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
In this section of the slice creation, we will add any additional software, tools, or scripts that we need for our experiments. For this slice, we only need to install "net-tools" and "traceroute". We provide the software by running node.execute() and letting the node know to download the software. Additionally, we will be removing the original routes between node A and node C. We use the "route flush" command to erase the available routes between these nodes. Disconnecting the routes will restrict how we will be sending data in our arrangement; the new format will look more like a line than a triangle in terms of network access. In terms of hardware, this would still be a triangle.

In [ ]:
# Install Software
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:        
    #remove existing routes for the experiment
    NDA.execute(f'sudo ip route flush dev {A_iface.get_device_name()}')
    NDC.execute(f'sudo ip route flush dev {C_iface.get_device_name()}')
    
    # install net-tools on each node
    for node in slice.get_nodes():
        #install route and traceroute
        node.execute("sudo apt update")
        node.execute("sudo apt install -y net-tools")
        node.execute("sudo apt install -y traceroute")
        
except Exception as e:
    print(f"Exception: {e}")

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the Routing IPV4 Notebook. You can either open it on the explorer or click [Here](./Routing_with_IPv4.ipynb) to open the next notebook.